<a href="https://colab.research.google.com/github/iryna-savchuk/CI4O_project/blob/experiments/CIFO_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
# Append Charles Library, we can do this by cloning the gitbug
# !git clone https://github.com/iryna-savchuk/CI4O_project.git

#import sys
#sys.path.append('/content/CI4O_project')

In [1]:
# Append Charles Library
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/CIFO_Project')

# Import Libraries

In [6]:
# Import charles library
from data.mnist_data import train_images, train_labels
from charles.charles import Population, Individual
from charles.selection import tournament_sel, fps
from charles.crossover import arithmetic_xo
from charles.mutation import inversion_mutation, arithmetic_mutation

In [7]:
# Import others libraries
import numpy as np
from tensorflow.keras import Sequential, layers
import csv
import sys

# Define Model and Fitness Function

In [8]:
def get_fitness(self):
    input = self.representation["input"]
    hidden = self.representation["hidden"]
    output = self.representation["output"]

    # Defining the architecture of a neural network
    nn = Sequential([
        layers.Dense(hidden, activation="relu", input_shape=(input,)),
        layers.Dense(output, activation="softmax")
    ])
    current_weights = np.array(self.representation["weights"])
    weights_1 = current_weights[:input * hidden].reshape(input, hidden)
    biases_1 = current_weights[input * hidden:input * hidden + hidden].reshape(hidden, )
    weights_2 = current_weights[input * hidden + hidden:input * hidden + hidden + hidden * output].reshape(hidden, output)
    biases_2 = current_weights[input * hidden + hidden + hidden * output:].reshape(output, )

    # Setting the weights of the model
    nn.set_weights([weights_1, biases_1, weights_2, biases_2])

    # Compiling the model
    nn.compile(loss='categorical_crossentropy', metrics=["accuracy"])

    train_loss, train_acc = nn.evaluate(train_images, train_labels)
    return train_acc

In [9]:
# Monkey patching
Individual.get_fitness = get_fitness

# Define funtion to run the model

In [10]:
def run_evolution(run, pop_size, gens, select, crossover, mutate, xo_prob, mut_prob, elitism, tourn_size, filename):
    for i in range(run):
        print(f"Running GA iteration: {i + 1}")
        # Generating Population
        pop = Population(
            size=pop_size,
            sol_input=28 * 28,
            sol_hidden=512,
            sol_output=10,
            valid_range=[-1, 1],
            optim="max")

        print(pop.individuals)
        # print(pop.individuals[0].representation)

        # Running evolution iterations
        print('Evolving...')
        pop.evolve(gens=gens,
                   select=select, mutate= mutate, crossover=crossover,
                   mut_prob=mut_prob, xo_prob=xo_prob,
                   elitism=elitism,
                   tourn_size=tourn_size)
        all_runs_best_fitness.append(pop.best_fitnesses)

    # Writing to CSV
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Best_fitness'])  # header
        for fitness in all_runs_best_fitness:
            writer.writerow([fitness])

    print(f"All best fitness, from each generation, from each run have been saved to {filename}")

# Running the models

## 1. Tournament Selection

## Tournament Size = 4

### Inversion_mutation with xo_prob 90% and mut_prob 30%

In [11]:
all_runs_best_fitness = []
run_evolution(run = 10, pop_size = 20, gens = 20, select = tournament_sel, crossover= arithmetic_xo, mutate= inversion_mutation,
              xo_prob= 0.9, mut_prob= 0.3, elitism= True, tourn_size= 4, filename = 'toun_4_inversion_90_30.csv')

Running GA iteration: 1
Generating initial population...


  0%|          | 0/20 [00:00<?, ?it/s]

1875/1875 [==============================] - 3s 2ms/step - loss: 73.7634 - accuracy: 0.1082


  5%|▌         | 1/20 [00:05<01:51,  5.87s/it]

1875/1875 [==============================] - 4s 2ms/step - loss: 62.6935 - accuracy: 0.1280


...

[
Individual: 784, 512, 10; Fitness: 0.10821666568517685, 
Individual: 784, 512, 10; Fitness: 0.12800000607967377, 
Individual: 784, 512, 10; Fitness: 0.09413333237171173, 
Individual: 784, 512, 10; Fitness: 0.08184999972581863, 
Individual: 784, 512, 10; Fitness: 0.09056666493415833, 
Individual: 784, 512, 10; Fitness: 0.05251666530966759, 
Individual: 784, 512, 10; Fitness: 0.15109999477863312, 
Individual: 784, 512, 10; Fitness: 0.1129833310842514, 
Individual: 784, 512, 10; Fitness: 0.08393333107233047, 
Individual: 784, 512, 10; Fitness: 0.1359499990940094, 
Individual: 784, 512, 10; Fitness: 0.07000000029802322, 
Individual: 784, 512, 10; Fitness: 0.12171666324138641, 
Individual: 784, 512, 10; Fitness: 0.0916166678071022, 
Individual: 784, 512, 10; Fitness: 0.11774999648332596, 
Individual: 784, 512, 10; Fitness: 0.0919666662812233, 
Individual: 784, 512, 10; Fitness: 0.10029999911785126, 
Individual: 784, 512, 10; Fitness: 0.13883332908153534, 
Individual: 784, 512, 10; Fitness

  0%|          | 0/20 [00:00<?, ?it/s]

Starting Generation  1 ...
 456/1875 [======>.......................] - ETA: 2s - loss: 51.9595 - accuracy: 0.1205

  0%|          | 0/20 [00:59<?, ?it/s]


KeyboardInterrupt: ignored

### Inversion_mutation with xo_prob 90% and mut_prob 10%

In [ ]:
all_runs_best_fitness = []
run_evolution(run = 10, pop_size = 20, gens = 20, select = tournament_sel, crossover= arithmetic_xo, mutate= inversion_mutation,
              xo_prob= 0.9, mut_prob= 0.3, elitism= True, tourn_size= 4, filename = 'toun_4_inversion_90_10.csv')

Running GA iteration: 1
313/313 [==============================] - 1s 2ms/step - loss: 2.7493 - accuracy: 0.8571
Gen 1, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.8727113008499146

313/313 [==============================] - 1s 2ms/step - loss: 2.4911 - accuracy: 0.8458
Gen 2, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6475142240524292

313/313 [==============================] - 1s 2ms/step - loss: 1.6219 - accuracy: 0.8783
Gen 3, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.621944546699524

313/313 [==============================] - 1s 2ms/step - loss: 2.7328 - accuracy: 0.8112
Gen 4, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.4994518756866455

313/313 [==============================] - 1s 2ms/step - loss: 1.7137 - accuracy: 0.8687
Gen 5, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.4404010772705078

313/313 [==================

### arithmetic_mutation with xo_prob 90% and mut_prob 30%

In [ ]:
all_runs_best_fitness = []
run_evolution(run = 10, pop_size = 20, gens = 20, select = tournament_sel, crossover= arithmetic_xo, mutate= arithmetic_mutation,
              xo_prob= 0.9, mut_prob= 0.3, elitism= True, tourn_size= 4, filename = 'toun_4_inversion_90_30.csv')

Running GA iteration: 1
313/313 [==============================] - 1s 2ms/step - loss: 2.8114 - accuracy: 0.8555
Gen 1, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6140621900558472

313/313 [==============================] - 1s 2ms/step - loss: 3.2359 - accuracy: 0.8194
Gen 2, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6140621900558472

313/313 [==============================] - 1s 2ms/step - loss: 2.2511 - accuracy: 0.8841
Gen 3, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6140621900558472

313/313 [==============================] - 1s 2ms/step - loss: 1.8008 - accuracy: 0.8968
Gen 4, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6140621900558472

313/313 [==============================] - 1s 2ms/step - loss: 4.2271 - accuracy: 0.8326
Gen 5, Best Individual: Individual:  (784, 512) (512,) (512, 10) (10,); Fitness: 1.6140621900558472

313/313 [=================

### arithmetic_mutation with xo_prob 90% and mut_prob 30%

In [ ]:
all_runs_best_fitness = []
run_evolution(run = 10, pop_size = 20, gens = 20, select = tournament_sel, crossover= arithmetic_xo, mutate= arithmetic_mutation,
              xo_prob= 0.9, mut_prob= 0.1, elitism= True, tourn_size= 4, filename = 'toun_4_inversion_90_30.csv')